In [1]:
import numpy as np
import pickle

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import save_params, load_params, printer_start, load_tool, unload_tool, tool_change, take_photo, play_sound, printer_stop
# from src.additional_functions import *
from src.network import Network_custom, replace_brackets
import os

BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"

In [2]:
# Note that the coordinates of the vertices will change after equilibrium is reached.
model_name  = '3D_structure_1'
vertices    = np.array([[-10, 0, 0], 
                        [-5, 0, 0], 
                        [5, 0, 0],
                        [10, 0, 0],
                        [0, 10, 0],
                        [0, 5, 0],
                        [0, -5, 0],
                        [0, -10, 0],
                        [-10, 0, 10], 
                        [-5, 0, 5], 
                        [5, 0, 5],
                        [10, 0, 10],
                        [0, 10, 10],
                        [0, 5, 5],
                        [0, -5, 5],
                        [0, -10, 10]])*10  # The coordinates of the vertices in mm
edges       = [[0, 1], [1, 6], [6, 2], [3, 2], [2, 5], [1, 5], [6, 7], [5, 4],
               [8, 9], [9, 14], [10, 14], [11, 10], [10, 13], [13, 9], [14, 15], [13, 12],
               [9, 1], [5, 13], [2, 10], [14, 6]]                  # The edges of the network.
paths       = [[0, 1, 2, 4 ,7], [8, 16, 5, 17, 15], [11, 12, 13, 9, 14], [3, 18, 10, 19, 6]]   # 3D printable paths. Ensure an edge is not flipped.]
directions  = [1, 1, 1, 1, 1, 1, 1, 1,
               1, 1, 1, 1, 1, 1, 1, 1,
               1, 1, 1, 1]          # Flip the curvature of an edge by setting the direction to -1.

q_ring      = .7
q_connectors = .5

q           = np.array([q_connectors, q_ring, q_ring, q_connectors, q_ring, q_ring, q_connectors, q_connectors,
                        q_connectors, q_ring, q_ring, q_connectors, q_ring, q_ring, q_connectors, q_connectors,
                        q_ring, q_ring, q_ring, q_ring])/20
fixed       = [0, 3, 4, 7, 8, 11, 12, 15]  # Fixed vertices

# Create the network
net = Network_custom.from_fd(vertices, edges, q, fixed, paths = paths, dir = directions)

# The topology of structure is fixed after generation, but you can update force densities after making the network
# q[0] = .10
# net.update_shape(q)

# Plot the network like this
net.net_plot(color=True, elables = True, vlabels = False)

(net.vertices == vertices).all() # Please understand that this is False

False

In [3]:
TPU = {'E':130, 'v':0.3897, 'p':1.18e-9, 'A':0.11*0.71, 'name': 'TPU conductive'} # Conductive TPU. Manufacturer Ninjatek Eel

net.set_material(TPU) # By saving the material properties in the network, we can easily see how the network was constructed.

# If all elements will get the same material, you can use the following line
E = [TPU['E']]*len(net.edges)
A = [TPU['A']]*len(net.edges)
l0, l_scalar = net.materialize(E, A)

# Yielding the initial lengths of each element. And the following scalar: min(l0/l1)
net.l0, net.l_scalar

c:\Users\thijs\Documents\GitHub\printing-tensioned-structures\src\network.py:207: SparseEfficiencyWarning:

spsolve requires A be CSC or CSR matrix format

c:\Users\thijs\Documents\GitHub\printing-tensioned-structures\src\network.py:207: SparseEfficiencyWarning:

spsolve is more efficient when sparse b is in the CSC matrix format



(array([68.48854816, 32.98445167, 32.98445167, 68.48854816, 32.98445167,
        32.98445167, 68.48854816, 68.48854816, 68.48854816, 32.98445167,
        32.98445167, 68.48854816, 32.98445167, 32.98445167, 68.48854816,
        68.48854816, 24.12704901, 24.12704901, 24.12704901, 24.12704901]),
 0.8313588393675654)

In [4]:
plane_normal = np.array([0.2, 0.7, 1])
net.get_net_projection(plane_normal)
net.net_plot(color=True, elables = False, vlabels = True, plot_type = 'projection')

In [5]:
net.initialize_shape_optimizer(function_type = 'standard',  method = 'L-BFGS-B')
net.optimize_vertices()
reference_point = [0,0,0]
net.scale_vertices(reference_point, net.l_scalar)
R, th = net.arc_param()
xyz = net.arc_points(n = 20) 
net.net_plot(color=True, elables = True, vlabels = False, plot_type = 'arcs')

In [6]:
# When no interpolation function is provided, linear interpolation is used. No other interpolation function is implemented (yet?).
# net.jump_at_intersection(intersection_width = 2, intersection_height = 2, interpolation_function=None) # These settings are just for visibility, this would be way to much

# These setting are more appropriate for printing
printing_params = load_params(r'DATA/NT_Eel_0.2mm_og.json')
net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 
net.get_crossings()
net.jump_at_crossings(crossing_width = printing_params['d_nozzle']*2, crossing_height = printing_params['layer_height']*3, interpolation_function = None)
net.net_plot(color=True, plot_type='arcs', elables=True, vlabels=False)

In [7]:
alpha_loop = np.deg2rad(6) # The angle of the loop
L_loop  = 11                # The length of the loop
n_points = 60               # The number of points in the loop

start_loop_bools = [True, True, True, True]
end_loop_bools = [True, True, True, True]

net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)

In [8]:
net.get_crossings()
net.jump_at_crossings( crossing_width = 1, crossing_height = 1, interpolation_function = None)

# crossings, net.crossings, net.intersections
len(net.xyz_vec), net.edges.shape[0]

(20, 20)

In [9]:
net.net_plot(color=True, plot_type='arcs', elables=True, vlabels=False)